In [1]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import pandas as pd
import torch

/home/hyj/anaconda3/envs/bert/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('../Classification_Sentence/Data/test_pred.csv',index_col=[0])
data.head(1)

,name,ad_label,summary,pred
0,371_분양계약,0,제8조(해약) 3. 을 의 준공지체 등의 사유로 콘도의 이용예정일로부터 5개월을 초...,0.0


In [3]:
pattern = '([제|0-9 ]{1,}조[\(|가-힣\) |A-Z|a-z|0-9|,m]{1,})'
for i in range(len(data)):
    text = data['summary'][i]
    matchedList = re.findall(pattern,text)
    for match in matchedList:
        text = text.replace(match,'')
    data['summary'][i] = text

pattern = '([제|0-9 ]{1,}항[\(|가-힣\) |A-Z|a-z|0-9|,m]{1,})'
for i in range(len(data)):
    text = data['summary'][i]
    matchedList = re.findall(pattern,text)
    for match in matchedList:
        text = text.replace(match,'')
    data['summary'][i] = text
data['summary'] = data['summary'].apply(lambda x: re.sub(r"[0-9]", "", x))

/home/hyj/anaconda3/envs/bert/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/hyj/anaconda3/envs/bert/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [4]:
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,ngram_range=(2,4), max_features=3000, binary=False, max_df=0.5,min_df=1)

good = data.loc[data['pred']==0]
good_x = vectorizer.fit_transform(good['summary'])

lda_model = LatentDirichletAllocation(n_components=15,learning_method='online',random_state=777,max_iter=15)
lda_top = lda_model.fit_transform(good_x)

terms = vectorizer.get_feature_names_out() 
def get_topics(components, feature_names, n=4):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i]) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: ['하는 경우', '있습니다 개정', '계약을 해지할', '소개로 상대방과']
Topic 2: ['받은 때에는', '경우에는 그러하지 아니하다', '그러하지 아니하다', '이후 해지할 경우']
Topic 3: ['있는 경우', '계약을 체결한', '전부 또는', '보증서발급일로부터 일이내에']
Topic 4: ['반복되거나 이내에 사유가 시정되지', '회원자격을 상실시킬', '반복되거나 이내에', '이상 반복되거나 이내에 사유가']
Topic 5: ['어느 하나에', '하나에 해당하는', '어느 하나에 해당하는', '다음 호의']
Topic 6: ['기간을 정하여', '이상의 기간을', '상실시키는 경우에는', '회원자격을 상실시키는 경우에는 회원등록을']
Topic 7: ['등의 사유로', '지체 없이', '청약을 철회할', '카드사는 회원이']
Topic 8: ['못할 경우에는', '이용자가 입실 예정일에', '예정일에 입실하지 못할', '예정일에 입실하지 못할 경우에는']
Topic 9: ['청구할 있습니다', '동의를 얻어야', '제회 보험료', '다만 고객이']
Topic 10: ['청약을 철회할', '철회할 있습니다', '별도로 정한 경우에는', '별도로 정한 경우에는 그에']
Topic 11: ['있도록 있습니다', '등에 관한', '있을 경우', '회사에 제공한']
Topic 12: ['다음 호에', '대부거래 표준약관', '계약을 해제할', '해제할 있습니다']
Topic 13: ['가입신청자가 약관', '통지하여야 합니다', '경우 회사는', '이전에 을에게 제공하여야 한다']
Topic 14: ['사정에 의하여', '이용금액의 퍼센트를', '수신자에게 불리하지', '수신자에게 불리하지 않은 경우']
Topic 15: ['원칙으로 합니다', '있는 경우에는', '초과하는 경우에는', '제공함을 원칙으로']


In [5]:
# 불리 문장
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,ngram_range=(2,5), max_features=3000, binary=False,min_df=2)
bad = data.loc[data['pred']==1]
bad_x = vectorizer.fit_transform(bad['summary'])

lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=10)
lda_top = lda_model.fit_transform(bad_x)

terms = vectorizer.get_feature_names_out() 
def get_topics(components, feature_names, n=4):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i]) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: ['보증금은 반환되지', '을이 부담한다', '한다 사용기간을 월단위로', '사용기간을 월단위로 정산하며']
Topic 2: ['홈페이지 안내', '이에 따른', '해지할 있다', '경우에는 별도의']
Topic 3: ['법원으로 한다', '제호 을이', '대한 이자는', '제호 갑의']
Topic 4: ['해당하는 금액을', '책임을 지지', '지지 않습니다', '정한 이외의 사항이 있거나 또는']
Topic 5: ['경우 갑은', '갑은 을에게', '청구할 있다', '내지 일전']
Topic 6: ['고의 또는', '경우에는 별도의', '제기하지 아니한다', '이의를 제기하지 아니한다']
Topic 7: ['고객 금융회사', '개월 이내', '환불 받을', '부가세 퍼센트']
Topic 8: ['회원권에 대해서는', '질병이 발생할', '책임도 지지', '책임도 지지 않습니다']
Topic 9: ['갑에게 귀속된다', '한다 경우', '연체료는 환불하지', '연체료는 환불하지 아니한다']
Topic 10: ['환불되지 않습니다', '서비스 이용', '예약금은 환불되지', '예약금은 환불되지 않습니다']
